In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.preprocessing.image import img_to_array, load_img

np.random.seed(2018)

Using TensorFlow backend.


In [4]:
def load_numeric_training(standardize=True):
    data = pd.read_csv('../train.csv')
    ID = data.pop('id')
    y = data.pop('species')
    y = LabelEncoder().fit(y).transform(y)
    X = StandardScaler().fit(data).transform(data) if standardize else data.values
    return ID, X, y
load_numeric_training();

In [5]:
def load_numeric_test(standardize=True):
    data = pd.read_csv('../test.csv')
    ID = data.pop('id')
    test = StandardScaler().fit(data).transform(data) if standardize else data.values
    return ID, test
load_numeric_test();

In [6]:
def resize_img(img, max_dim=96):
    max_axis = np.argmax(img.size)
    scale = max_dim / img.size[max_axis]
    return img.resize((int(img.size[0] * scale), int(img.size[1] * scale)))


def load_img_data(ids, max_dim=96, center=True):
    X = np.empty((len(ids), max_dim, max_dim, 1))
    for i, id in enumerate(ids):
        img = load_img('../images/{}.jpg'.format(id), grayscale=True)
        img = resize_img(img, max_dim=max_dim)
        x = img_to_array(img)
        h, w = x.shape[:2]
        if center:
            h1 = (max_dim - h) >> 1
            h2 = h1 + h
            w1 = (max_dim - w) >> 1
            w2 = w1 + w
        else:
            h1, h2, w1, w2 = 0, h, 0, w
        X[i][h1:h2, w1:w2][:] = x
    return np.around(X / 255)


In [7]:
def load_train_data(split=0.9, random_state=7):
    ID, X_num_train, y = load_numeric_training()
    X_img_train = load_img_data(ID)
    sss = StratifiedShuffleSplit(n_splits=1, train_size=split, test_size=1 - split, random_state=random_state)
    train_idx, val_idx = next(sss.split(X_num_train, y))
    X_num_tr, X_img_tr, y_tr = X_num_train[train_idx], X_img_train[train_idx], y[train_idx]
    X_num_val, X_img_val, y_val = X_num_train[val_idx], X_img_train[val_idx], y[val_idx]
    return (X_num_tr, X_img_tr, y_tr), (X_num_val, X_img_val, y_val)


def load_test_data():
    ID, X_num_test = load_numeric_test()
    X_img_test = load_img_data(ID)
    return ID, X_num_test, X_img_test


In [8]:
(X_num_tr, X_img_tr, y_tr), (X_num_val, X_img_val, y_val) = load_train_data()
ID_test, X_num_test, X_img_test = load_test_data()
y_tr, y_val = to_categorical(y_tr), to_categorical((y_val))

In [9]:
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator, array_to_img


class ImageDataGenerator2(ImageDataGenerator):
    def __init__(self, rotation_range=20, zoom_range=0.2, horizontal_flip=True,
                 vertical_flip=True, fill_mode='nearest', X_num_tr=X_num_tr):
        super().__init__(rotation_range=20, zoom_range=0.2, horizontal_flip=True,
                         vertical_flip=True, fill_mode='nearest')
        self.X_num_tr = X_num_tr

    def flow(self, X, y=None, batch_size=32, shuffle=True, seed=None,
             save_to_dir=None, save_prefix='', save_format='jpeg'):
        return NumpyArrayIterator2(X, y, self,
                                   batch_size=batch_size, shuffle=shuffle, seed=seed,
                                   save_to_dir=save_to_dir, save_prefix=save_prefix,
                                   save_format=save_format, X_num_tr=self.X_num_tr)


class NumpyArrayIterator2(NumpyArrayIterator):
    def __init__(self, X, y, generator, batch_size, shuffle, seed,
                 save_to_dir, save_prefix, save_format, X_num_tr):
        super().__init__(X, y, generator, batch_size, shuffle, seed,
                         save_to_dir, save_prefix, save_format)
        self.X_num_tr = X_num_tr

    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        # Keeps under lock only the mechanism which advances
        # the indexing of each batch.
        with self.lock:
            index_array = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel
        batch_X, batch_y = self._get_batches_of_transformed_samples(index_array)
        return {'image': batch_X, 'numerical': self.X_num_tr[index_array]}, batch_y


imgen = ImageDataGenerator2(rotation_range=20, zoom_range=0.2, horizontal_flip=True,
                            vertical_flip=True, fill_mode='nearest', X_num_tr=X_num_tr)
imgen_train = imgen.flow(X_img_tr, y_tr, batch_size=32, seed=7)


In [10]:
from keras.layers import Dense, Dropout, Activation, Convolution2D, MaxPooling2D, \
    Flatten, Input, concatenate
from keras.models import Model


def combined_model():
    image = Input(shape=(96, 96, 1), name='image')
    x = Convolution2D(8, (5, 5), input_shape=(96, 96, 1))(image)
    x = (Activation('relu'))(x)
    x = (MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))(x)

    x = Convolution2D(32, (5, 5))(x)
    x = Activation('relu')(x)
    x = (MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))(x)
    x = Flatten()(x)

    numerical = Input(shape=(192,), name='numerical')
    concatenated = concatenate([x, numerical])

    x = Dense(100, activation='relu')(concatenated)
    x = Dropout(0.5)(x)

    out = Dense(99, activation='softmax')(x)

    model = Model(inputs=[image, numerical], outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


def image_model():
    image = Input(shape=(96, 96, 1), name='image')
    x = Convolution2D(8, (5, 5), input_shape=(96, 96, 1))(image)
    x = (Activation('relu'))(x)
    x = (MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))(x)

    x = Convolution2D(32, (5, 5))(x)
    x = Activation('relu')(x)
    x = (MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))(x)
    x = Flatten()(x)

    x = Dense(100, activation='relu')(x)
    x = Dropout(0.5)(x)

    out = Dense(99, activation='softmax')(x)

    model = Model(inputs=image, outputs=out)
    model.compile(optimizer=optimizers.Adam(lr=0.002), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [11]:
model = combined_model()
model2 = image_model()

In [176]:
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

def combined_generator(imgen):
    while True:
        for i in range(891 // 32):
            batch_x, batch_y = next(imgen)
            while len(imgen.index_array)==0:
                batch_x, batch_y = next(imgen)
            yield batch_x, batch_y


best_model_file = 'leafnet.h5'
best_model = ModelCheckpoint(best_model_file, monitor='val_loss', verbose=1,
                             save_best_only=True)

history = model.fit_generator(combined_generator(imgen_train),
                              steps_per_epoch=len(X_img_tr) // 32,
                              epochs=99, validation_data=[{'image': X_img_val, 'numerical': X_num_val}, y_val]
                              , verbose=1, callbacks=[best_model])

Epoch 1/99
27/27 [==============================] - 7s 252ms/step - loss: 4.3923 - acc: 0.0694 - val_loss: 3.8706 - val_acc: 0.3333
Epoch 2/99
27/27 [==============================] - 4s 132ms/step - loss: 3.6254 - acc: 0.1865 - val_loss: 2.7781 - val_acc: 0.6061
Epoch 3/99
27/27 [==============================] - 4s 131ms/step - loss: 2.8788 - acc: 0.3171 - val_loss: 1.8186 - val_acc: 0.8384
Epoch 4/99
27/27 [==============================] - 4s 131ms/step - loss: 2.2633 - acc: 0.4318 - val_loss: 1.2295 - val_acc: 0.8687
Epoch 5/99
27/27 [==============================] - 4s 133ms/step - loss: 1.7343 - acc: 0.5693 - val_loss: 0.8413 - val_acc: 0.9293
Epoch 6/99
27/27 [==============================] - 4s 134ms/step - loss: 1.4202 - acc: 0.6182 - val_loss: 0.6228 - val_acc: 0.9798
Epoch 7/99
27/27 [==============================] - 4s 132ms/step - loss: 1.1127 - acc: 0.7222 - val_loss: 0.4526 - val_acc: 0.9798
Epoch 8/99
27/27 [==============================] - 4s 138ms/step - loss: 0.

Epoch 30/99
27/27 [==============================] - 3s 130ms/step - loss: 0.2008 - acc: 0.9512 - val_loss: 0.0304 - val_acc: 1.0000
Epoch 31/99
27/27 [==============================] - 3s 129ms/step - loss: 0.1477 - acc: 0.9706 - val_loss: 0.0325 - val_acc: 1.0000
Epoch 32/99
27/27 [==============================] - 4s 131ms/step - loss: 0.1555 - acc: 0.9647 - val_loss: 0.0266 - val_acc: 1.0000
Epoch 33/99
27/27 [==============================] - 3s 129ms/step - loss: 0.1720 - acc: 0.9531 - val_loss: 0.0267 - val_acc: 1.0000
Epoch 34/99
27/27 [==============================] - 4s 130ms/step - loss: 0.1602 - acc: 0.9641 - val_loss: 0.0255 - val_acc: 1.0000
Epoch 35/99
27/27 [==============================] - 3s 129ms/step - loss: 0.1687 - acc: 0.9653 - val_loss: 0.0293 - val_acc: 0.9899
Epoch 36/99
27/27 [==============================] - 3s 128ms/step - loss: 0.1475 - acc: 0.9630 - val_loss: 0.0278 - val_acc: 0.9899
Epoch 37/99
27/27 [==============================] - 3s 129ms/step - 

Epoch 62/99
27/27 [==============================] - 4s 159ms/step - loss: 0.0805 - acc: 0.9792 - val_loss: 0.0141 - val_acc: 1.0000
Epoch 63/99
27/27 [==============================] - 4s 160ms/step - loss: 0.0965 - acc: 0.9734 - val_loss: 0.0188 - val_acc: 1.0000
Epoch 64/99
27/27 [==============================] - 4s 164ms/step - loss: 0.0843 - acc: 0.9755 - val_loss: 0.0180 - val_acc: 0.9899
Epoch 65/99
27/27 [==============================] - 4s 153ms/step - loss: 0.0679 - acc: 0.9847 - val_loss: 0.0117 - val_acc: 1.0000
Epoch 66/99
27/27 [==============================] - 4s 151ms/step - loss: 0.0763 - acc: 0.9803 - val_loss: 0.0124 - val_acc: 1.0000
Epoch 67/99
27/27 [==============================] - 4s 150ms/step - loss: 0.0741 - acc: 0.9757 - val_loss: 0.0164 - val_acc: 1.0000
Epoch 68/99
27/27 [==============================] - 4s 139ms/step - loss: 0.0602 - acc: 0.9847 - val_loss: 0.0119 - val_acc: 1.0000
Epoch 69/99
27/27 [==============================] - 4s 139ms/step - 

Epoch 94/99
27/27 [==============================] - 4s 134ms/step - loss: 0.0502 - acc: 0.9884 - val_loss: 0.0067 - val_acc: 1.0000
Epoch 95/99
27/27 [==============================] - 4s 133ms/step - loss: 0.0545 - acc: 0.9884 - val_loss: 0.0075 - val_acc: 1.0000
Epoch 96/99
27/27 [==============================] - 4s 134ms/step - loss: 0.0317 - acc: 0.9963 - val_loss: 0.0072 - val_acc: 1.0000
Epoch 97/99
27/27 [==============================] - 4s 135ms/step - loss: 0.0314 - acc: 0.9942 - val_loss: 0.0076 - val_acc: 1.0000
Epoch 98/99
27/27 [==============================] - 4s 134ms/step - loss: 0.0310 - acc: 0.9931 - val_loss: 0.0062 - val_acc: 1.0000
Epoch 99/99
27/27 [==============================] - 4s 137ms/step - loss: 0.0343 - acc: 0.9873 - val_loss: 0.0049 - val_acc: 1.0000
